In [ ]:
from pathlib import Path

import torch
from datasets import Dataset, DatasetDict, load_dataset
from torch import Tensor, nn
from tqdm import tqdm

from luminar.classifier import LuminarClassifier
from luminar.encoder import LuminarEncoder
from luminar.utils import get_matched_datasets

HF_TOKEN = (Path.home() / ".hf_token").read_text().strip()

In [ ]:
from typing import Iterable

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader

from luminar.utils import PaddingDataloader


def get_dataloader(
    datset: Dataset | Iterable[dict[str, Tensor]],
    feature_dim: tuple[int, ...],
    batch_size: int = 32,
    **kwargs,
) -> DataLoader:
    return PaddingDataloader(
        dataset,
        feature_dim=feature_dim,
        batch_size=batch_size,
        **kwargs,
    )

### Encoder

In [ ]:
# encoder = LuminarEncoder()
# encoder.device = "cuda:0"

### Classifier

In [ ]:
agent = "gpt_4o_mini"
config = {
    # first 256 features & 13 layers for gpt2
    "feature_dim": (256, 13),
    "feature_type": "intermediate_likelihoods",
    "feature_selection": "first",
    "projection_dim": 32,
    "learning_rate": 0.0001,
    "warmup_steps": 66,
    "max_epochs": 25,
    "gradient_clip_val": 1.0,
    "batch_size": 32,
    "seed": 42,
    "agent": agent,
}
feature_len = config["feature_dim"][0]

In [ ]:
dataset: Dataset = (
    load_dataset(
        "liberi-luminaris/PrismAI-encoded-gpt2",
        "cnn_news-fulltext",
        split=f"human+{agent}",
        token=HF_TOKEN,
    )  # type: ignore
    .map(
        lambda features: {"features": features[:feature_len]},
        input_columns=["features"],
        desc="Trimming Features",
    )
    .rename_column("label", "labels")
)

In [33]:
datasets_matched = get_matched_datasets(dataset, agent).with_format(
    "torch", columns=["labels", "features"]
)
datasets_matched

DatasetDict({
    train: Dataset({
        features: ['agent', 'id_sample', 'id_source', 'labels', 'length', 'features'],
        num_rows: 18902
    })
    test: Dataset({
        features: ['agent', 'id_sample', 'id_source', 'labels', 'length', 'features'],
        num_rows: 4726
    })
    test_unmatched: Dataset({
        features: ['agent', 'id_sample', 'id_source', 'labels', 'length', 'features'],
        num_rows: 6959
    })
})

In [34]:
dataset_train = datasets_matched["train"].train_test_split(
    test_size=1 / 8,
    shuffle=True,
    seed=config["seed"],
).with_format(
    "torch", columns=["labels", "features"]
)

In [ ]:
dataloader_val = get_dataloader(
    dataset_train["test"],
    feature_dim=config["feature_dim"],
    batch_size=config["batch_size"],
)
dataloader_test = get_dataloader(
    datasets_matched["test"],
    feature_dim=config["feature_dim"],
    batch_size=config["batch_size"],
)
dataloader_test_unmatched = get_dataloader(
    datasets_matched["test_unmatched"],
    feature_dim=config["feature_dim"],
    batch_size=config["batch_size"],
)

In [17]:
seed_everything(config["seed"], verbose=False)
classifier = LuminarClassifier(**config)
trainer = Trainer(
    max_epochs=config["max_epochs"],
    logger=pl_loggers.TensorBoardLogger(
        save_dir=f"logs/in_domain/{agent}",
        name="cnn_news",
    ),
    gradient_clip_val=config["gradient_clip_val"],
    callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=3)],
    deterministic=True,
)
trainer.progress_bar_callback.disable()

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
dataloader_train = get_dataloader(
    dataset_train["train"].to_list(),
    feature_dim=config["feature_dim"],
    batch_size=config["batch_size"],
    shuffle=True,
)
trainer.fit(
    classifier,
    dataloader_train,
    dataloader_val,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name        | Type              | Params | Mode  | In sizes      | Out sizes  
----------------------------------------------------------------------------------------
0 | conv_layers | Sequential        | 152 K  | train | [32, 13, 256] | [32, 16384]
1 | projection  | Sequential        | 524 K  | train | [32, 16384]   | [32, 32]   
2 | classifier  | Linear            | 33     | train | [32, 32]      | [32, 1]    
3 | criterion   | BCEWithLogitsLoss | 0      | train | ?             | ?          
----------------------------------------------------------

TypeError: expected Tensor as element 0 in argument 0, but got list

In [ ]:
metrics = trainer.test(
    classifier, (dataloader_test, dataloader_test_unmatched), verbose=False
)
metrics

In [ ]:
import gc
import json

from tqdm.auto import tqdm

scores = {}
for config_name, dataset in tqdm(datasets_truncated.items(), desc="Training Models"):
    model = LuminarClassifier()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)

    dataset_train = dataset["train"].with_format("torch", ["features", "label"])

    tq = tqdm(range(3), desc="Training " + config_name, leave=False)
    for i in tq:
        for batch in dataset_train.shuffle(i).batch(batch_size=32):
            optimizer.zero_grad()
            features = batch["features"]
            labels = batch["label"].float().unsqueeze(-1)

            preds = model(features)

            loss = criterion(preds, labels)

            loss.backward()
            optimizer.step()

            tq.set_postfix_str(f"loss: {loss.item()}")

    model.eval()
    scores[config_name] = evaluate(model, {config_name: dataset})[config_name]
    ## OOD Evaluation
    # scores[config_name] = evaluate(model, datasets_truncated)

    print(config_name, json.dumps(scores[config_name], indent=4))

    del model
    gc.collect()
    torch.cuda.empty_cache()


In [ ]:
raise RuntimeError()

In [ ]:
import json

print(json.dumps(scores, indent=4))
with open("../logs/luminar/gpt2_first_128-3_epochs.json", "w") as f:
    json.dump(scores, f, indent=4)

In [ ]:
datasets = {}
for subset in [
    "blog_authorship_corpus",
    "student_essays",
    "cnn_news",
    "euro_court_cases",
    "house_of_commons",
    "arxiv_papers",
    "gutenberg_en",
    "en",
    "bundestag",
    "spiegel_articles",
    "gutenberg_de",
    "de",
]:
    config_name = f"{subset}-fulltext"
    datasets[config_name] = load_dataset(
        "liberi-luminaris/PrismAI-encoded-gpt2",
        config_name,
        token=HF_TOKEN,
        split="human+gpt_4o_mini",
    )

In [ ]:
dataset = load_dataset(
    "liberi-luminaris/PrismAI-fulltext", "cnn_news", split="human+gpt_4o_mini"
)
dataset_human = dataset.filter(lambda sample: sample["agent"] == "human")
source_ids = set(
    dataset_human.shuffle(seed=42).take(len(dataset_human) // 10 * 8)["id_source"]
)
dataset_train = dataset.filter(lambda sample: sample["id_source"] in source_ids)

In [ ]:
datasets_truncated = {}
for config_name, dataset in datasets.items():
    datasets_truncated[config_name] = dataset.with_format(
        "numpy", columns=["features"], output_all_columns=True
    ).map(
        lambda batch: {"features": batch["features"][:, :256]},
        batched=True,
    )

In [ ]:
datasets_considered = {
    key: value
    for key, value in datasets_truncated.items()
    if not key.startswith("de-") and not key.startswith("en-")
}

In [ ]:
import pandas as pd

domains = [
    "Web Blogs",
    "Essays",
    "CNN",
    "ECHR",
    "HoC",
    "arXiv",
    "Gutenberg$_{en}$",
    "Bundestag$_{de}$",
    "Spiegel$_{de}$",
    "Gutenberg$_{de}$",
    "All$_{en}$",
    "All$_{de}$",
]
name_map = {
    "blog_authorship_corpus": "Web Blogs",
    "student_essays": "Essays",
    "cnn_news": "CNN",
    "euro_court_cases": "ECHR",
    "house_of_commons": "HoC",
    "arxiv_papers": "arXiv",
    "gutenberg_en": "Gutenberg$_{en}$",
    "bundestag": "Bundestag$_{de}$",
    "spiegel_articles": "Spiegel$_{de}$",
    "gutenberg_de": "Gutenberg$_{de}$",
    "en": "All$_{en}$",
    "de": "All$_{de}$",
}

results = [
    {"domain": name_map[key.split("-", 1)[0]]}
    | {
        "f1": value["f1"],
        "acc": value["accuracy"],
        "auroc": value["auroc"],
    }
    for key, value in scores.items()
]
metric_df = (
    pd.DataFrame(results)
    .set_index("domain")
    .sort_index(key=lambda x: list(map(domains.index, x)))
)
print(metric_df.to_latex(float_format="%.3f", index=True))
metric_df

In [ ]:
# def run_detector(
#     detector: DetectorABC, datasets: dict[str, DatasetDict]
# ) -> dict[str, float]:
#     scores = {}
#     for config_name, ds in tqdm(datasets.items(), desc="Predicting on Datasets"):
#         dataset: Dataset = ds["test"].map(
#             detector.tokenize,
#             input_columns=["text"],
#             batched=True,
#             batch_size=1024,
#             desc="Tokenizing",
#         )
#         dataset = dataset.sort("length")
#         dataset = dataset.map(
#             detector.process,
#             batched=True,
#             batch_size=128,
#             desc="Predicting",
#         )

#         dataset_np = dataset.select_columns(["prediction", "label"]).with_format(
#             "numpy"
#         )

#         acc, f1, auroc = get_scores(dataset_np["label"], dataset_np["prediction"])
#         scores[config_name] = {"accuracy": acc, "f1": f1, "auroc": auroc}

#         acc, f1, auroc = get_scores(
#             dataset_np["label"],
#             dataset_np["prediction"],
#             calibrated=True,
#         )
#         scores[config_name] |= {
#             "accuracy_calibrated": acc,
#             "f1_calibrated": f1,
#             "auroc_calibrated": auroc,
#         }
#     return scores


In [ ]:
# def evaluate(model: LuminarClassifier, datasets: dict[str, DatasetDict]) -> dict:
#     scores = {}
#     for config_name, dataset in tqdm(datasets.items(), desc="Evaluating", leave=False):
#         ds = (
#             dataset["test"]
#             .with_format("torch", ["features"])
#             .map(model.process, batched=True, batch_size=32, desc="Predicting")
#         )
#         dataset_np = ds.select_columns(["prediction", "label"]).with_format("numpy")

#         acc, f1, auroc = get_scores(dataset_np["label"], dataset_np["prediction"])
#         scores[config_name] = {
#             "accuracy": acc,
#             "f1": f1,
#             "auroc": auroc,
#         }

#         acc, f1, auroc = get_scores(
#             dataset_np["label"],
#             dataset_np["prediction"],
#             calibrated=True,
#         )
#         scores[config_name] |= {
#             "accuracy_calibrated": acc,
#             "f1_calibrated": f1,
#             "auroc_calibrated": auroc,
#         }

#     return scores